In [1]:
import numpy as np
import analysis
s = np.array([0, 0, 1, 0, 2, 2, 2])
n = 3
m = 4
seq= np.array([[0, 1, 1, 1],
               [0, 0, 1, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [1, 0, 0, 1],
               [1, 1, 0, 0],
               [0, 1, 0, 1]])

hmm = analysis.HMM4binary_sensors()
hmm.fit_with_true_hidden_states(seq, s)
print(hmm.predict_states(seq))
print(hmm.A)
print(hmm.P)

(array([0, 1, 0, 2, 2, 2, 2]), 5.873606637142921e-07)
[[0.33333333 0.33333333 0.33333333]
 [1.         0.         0.        ]
 [0.         0.         1.        ]]
[[0.         0.66666667 0.66666667 0.33333333]
 [0.         0.         1.         0.        ]
 [0.66666667 0.66666667 0.         0.66666667]]


In [3]:
import numpy as np
import analysis
C = np.array([1/2, 1/2])
A = np.array([[1/4, 3/4],
              [1/4, 3/4]])
P = np.array([[4/5, 1/5],
              [1/5, 2/3]])
s = np.random.randint(0, 2)
states = [s]
data = [[np.random.binomial(1, P[s][0]), np.random.binomial(1, P[s][1])]]
for i in range(10000):
    flag = np.random.binomial(1, A[s][0])
    if s == 0:        
        if not(flag):
            s = 1
    else:
        if flag:
            s = 0
    states.append(s)
    data.append([np.random.binomial(1, P[s][0]), np.random.binomial(1, P[s][1])])
states = np.array(states)
data = np.array(data)

hmm = analysis.HMM4binary_sensors()
hmm.fit_with_true_hidden_states(data, states)
print(hmm.C)
print(hmm.A)
print(hmm.P)

[0. 1.]
[[0.25408204 0.74591796]
 [0.25023368 0.74976632]]
[[0.79299363 0.21218153]
 [0.20269729 0.67071705]]


In [8]:
import numpy as np

C = np.array([0.9, 0.1])
P = np.array([[0.5, 0.5, 0.5, 0.5],
              [0.1, 0.1, 0.9, 0.9]])
o_t = np.array([0, 0, 1, 1])
ones_vec = np.ones(o_t.shape[0])
L = [C[i] * np.prod(P[i]*o_t + (ones_vec - P[i])*np.abs((ones_vec - o_t))) for i in range(P.shape[0])]
print(L)

[0.05625, 0.06561000000000002]


In [17]:
import numpy as np
def logsub(logX, logY):
        """
        This calculates log(X - Y) from log(X) and log(Y).
        
        Parameters
        ----------
        logX : float
        logY : float
        
        Returns
        -------
        ret : float
        """
        if logX < logY:
            raise ValueError('logX must be larger than logY')
        if logX == float('inf'):
            return logX
        diff = logY - logX
        if diff < -20:
            return logX
        return logX + np.log(1 - np.exp(diff))

n = 3
m = 2
logC = np.array([-1, -2.4, -3])
P = np.array([[0, 1],
              [0.3, 0.7],
              [0.05, 0.95]])
logP = np.log(P)
log1P = np.array([[logsub(0, logP[i][j]) for j in range(m)] for i in range(n)])
o_t = np.array([0, 1])
ones_vec = np.array([1, 1])

logp = [logC[i] + np.sum(logP[i]*o_t + log1P[i]*(ones_vec - o_t)) for i in range(n)]
ret = np.argmax(logp)
prob = np.max(logp)

logp_a = logC + np.sum(np.where(np.repeat(o_t[np.newaxis, :], n, axis=0), logP, log1P), axis = 1)
ret_a = np.argmax(logp_a)
prob_a = np.max(logp_a)

print(ret)
print(prob)
print(logp)
print(logp_a)
print(ret == ret_a)
print(prob == prob_a)

0
nan
[nan, -3.113349887877465, -3.102586588775101]
[-1.         -3.11334989 -3.10258659]
True
False


C:\Users\tanakai\AppData\Local\Temp\ipykernel_7476\2728804073.py:30: RuntimeWarning: divide by zero encountered in log
  logP = np.log(P)
C:\Users\tanakai\AppData\Local\Temp\ipykernel_7476\2728804073.py:22: RuntimeWarning: divide by zero encountered in log
  return logX + np.log(1 - np.exp(diff))
C:\Users\tanakai\AppData\Local\Temp\ipykernel_7476\2728804073.py:35: RuntimeWarning: invalid value encountered in multiply
  logp = [logC[i] + np.sum(logP[i]*o_t + log1P[i]*(ones_vec - o_t)) for i in range(n)]


In [21]:
a = np.array([[1, 0], [0, 1]])
b = np.array([[1, 2], [3, 4]])
c = np.array([[9, 8], [7, 6]])
np.where(a, b, c)

array([[1, 8],
       [7, 4]])

In [20]:
a = np.array([1,2,3])
b = a[np.newaxis, :]
print(b)

[[1 2 3]]


In [33]:
a = np.arange(5).reshape((5, 1))
b = np.arange(25).reshape((5, 5))
print(a)
print(b)
print((a + b))
print(np.max(a + b, axis = 0))
print(np.argmax(a + b, axis = 0))

[[0]
 [1]
 [2]
 [3]
 [4]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[ 0  1  2  3  4]
 [ 6  7  8  9 10]
 [12 13 14 15 16]
 [18 19 20 21 22]
 [24 25 26 27 28]]
[24 25 26 27 28]
[4 4 4 4 4]
